## Using with OpenFE

Here's how you would typically use these paths with OpenFE to load the actual molecular structures:

In [1]:
from openfe import ProteinComponent, SmallMoleculeComponent, LigandNetwork
from rdkit import Chem

from openfe_benchmarks.data import get_benchmark_data_system

Load a benchmark system that has network files. 
Using HNE from mcs_docking_set which has networks, cofactors, and charged ligands

In [2]:
system = get_benchmark_data_system("mcs_docking_set", "hne")

2026-02-04 13:49:57 | INFO     | Loaded system 'hne' from benchmark set 'mcs_docking_set' with 5 ligand file(s), and 5 cofactor file(s).
Found protein file: True.
Found 1 ligand network files


In [3]:
# Load protein
protein = ProteinComponent.from_pdb_file(str(system.protein))
print(f"Loaded protein: {protein}")
print(f"  Number of atoms: {protein.to_rdkit().GetNumAtoms()}")

Loaded protein: ProteinComponent(name=)
  Number of atoms: 3804


In [4]:
# Check available ligand files
print(f"\nAvailable ligand files for {system.name}:")
for charge_type in system.ligands.keys():
    print(f"  - {charge_type}")

# Load ligands without charges first (most reliable)
# Then OpenFE can assign charges during setup
print(f"\nLoading ligands from: {system.ligands['no_charges'].name}")

ligand_supplier = Chem.SDMolSupplier(str(system.ligands["no_charges"]), removeHs=False)
ligands = []
for i, mol in enumerate(ligand_supplier):
    if mol is not None:
        try:
            ligands.append(SmallMoleculeComponent(mol))
        except Exception as e:
            print(f"  Warning: Skipping molecule {i + 1} due to error: {e}")

print(f"\nSuccessfully loaded {len(ligands)} ligands")

# Show details of first few ligands
print("\nFirst 3 ligands:")
for i, ligand in enumerate(ligands[:3], 1):
    rdkit_mol = ligand.to_rdkit()
    print(
        f"  {i}. {ligand.name} - {rdkit_mol.GetNumAtoms()} atoms, {rdkit_mol.GetNumBonds()} bonds"
    )


Available ligand files for hne:
  - openeye_am1bcc
  - nagl_openff-gnn-am1bcc-1.0.0.pt
  - no_charges
  - antechamber_am1bcc
  - openeye_am1bccelf10

Loading ligands from: ligands.sdf

Successfully loaded 17 ligands

First 3 ligands:
  1. 6 - 55 atoms, 58 bonds
  2. 14 - 49 atoms, 51 bonds
  3. 15 - 56 atoms, 58 bonds


In [5]:
# Load network if available
network_path = system.ligand_networks["industry_benchmarks_network"]

print(f"\n{'=' * 60}")
print(f"Loading network from: {network_path.name}")

# Load network using from_json with file parameter (pass the file path directly)
network = LigandNetwork.from_json(file=str(network_path))

print("  Network loaded successfully!")
print(f"  Network nodes: {len(network.nodes)}")
print(f"  Network edges: {len(network.edges)}")

# Show some example edges
print("\nFirst 3 network edges:")
for i, edge in enumerate(list(network.edges)[:3], 1):
    print(f"  {i}. {edge.componentA.name} <-> {edge.componentB.name}")

# Note: The network contains its own SmallMoleculeComponent objects
# which may differ slightly from the ligands we loaded above
print(
    f"\nNote: Network contains {len(network.nodes)} ligand(s) with embedded molecule data"
)


Loading network from: industry_benchmarks_network.json
  Network loaded successfully!
  Network nodes: 17
  Network edges: 23

First 3 network edges:
  1. 14 <-> 15
  2. 6 <-> 18
  3. 28 <-> 30

Note: Network contains 17 ligand(s) with embedded molecule data


In [6]:
# Load cofactors if available
cofactors = []
if system.cofactors:
    print(f"\n{'=' * 60}")
    print(f"Loading cofactors from: {system.cofactors['no_charges'].name}")
    cofactor_supplier = Chem.SDMolSupplier(
        str(system.cofactors["no_charges"]), removeHs=False
    )
    for mol in cofactor_supplier:
        if mol is not None:
            print(mol)
            try:
                cofactors.append(
                    SmallMoleculeComponent(mol, name=mol.GetProp("s_m_entry_name"))
                )
            except Exception as e:
                print(f"  Warning: Skipping cofactor due to error: {e}")
    print(f"Successfully loaded {len(cofactors)} cofactor(s)")


Loading cofactors from: cofactors.sdf
Successfully loaded 1 cofactor(s)


/Users/jenniferclark/bin/gufe/gufe/components/explicitmoleculecomponent.py:27: UserWarning: Component being renamed from EPE-1 to Structure11.
  warnings.warn(f"Component being renamed from {rdkit_name} to {name}.")


In [7]:
print(f"\n{'=' * 60}")
print("System components ready for OpenFE:")
print(
    f"  Protein: {protein.name if protein.name else 'unnamed'} ({system.protein.name})"
)
print(f"  Ligands: {len(ligands)} molecules loaded from SDF")
print(f"  Cofactors: {len(cofactors)} molecule(s)")
print(f"  Network: {len(network.edges) if network else 0} edges")


System components ready for OpenFE:
  Protein: unnamed (protein.pdb)
  Ligands: 17 molecules loaded from SDF
  Cofactors: 1 molecule(s)
  Network: 23 edges
